# Planteo

En esta práctica vamos a ver en detalle como funciona SGD. Comenzamos con modelos lineales simples para resolver un problema de ejemplo con el objetivo de entender como funcionan los algoritmos sobre los que después vamos a construir redes complejas. 

El problema puntual es el siguiente:

Tenemos varias mediciones de temperatura en grados celsius y la misma cantidad de mediciones en una unidad desconocida (tomadas por un termómetro que no indica la unidad). Queremos ver si podemos encontrar una función que transforme de esta unidad desconocida a grados celsius.

***

Las medidas tomadas son las siguientes:

Grados celsius: [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]

Equivalente en unidad desconocida: [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]


In [ ]:
import torch
import numpy as np

inputs = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4])  # Datos observados
targets = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0])     # Equivalente en celsius

inputs = inputs / 10  # Normalizamos los datos para facilitar entrenamiento.

Definimos un modelo lineal (regresion) como una función, la misma recibe el input (X), los pesos (W) y los bias (b). 

Al ser una regresión lineal, el modelo simplemente computa el producto de cada peso por el input y le suma el bias (No es una red, no tiene activacion!)


In [ ]:
def model(X, w, b):
  return w * X + b

In [ ]:
# Como usamos el modelo

w_test = torch.ones(())
b_test = torch.zeros(())

prediccion = model(inputs, w_test, b_test)
prediccion

Vamos a usar MSE (Mean Squared Error - Error Cuadrático Medio) como función de costo

In [ ]:
def loss_fn(predictions, true_y):
  # ??

loss_fn(prediccion, targets)

# Actualizando

Queremos que el modelo minimize nuestra función de costo, para esto vamos a hacer uso de gradient descent.

GD nos dice que una actualización para nuestros pesos esta dada por:

***

![Image](https://blog.paperspace.com/content/images/2018/05/sgd.png)
***

Donde alfa es un learning rate en el rango (0-1) y es multiplicado por la derivada del costo con respecto a los pesos. En este caso, necesitamos el gradiente con respecto a los pesos W y el bias B.


![Image](https://i.ibb.co/xDW81qF/asd.png)




In [ ]:
# Por la regla de la cadena: 
# dloss/dw = (d loss / d targets) * (d targets / d w)
# dloss/db = (d loss / d targets) * (d targets / d b)

def d_loss_d_targets(predictions, targets):
    # x^2' = 2 * x

def d_model_dw(X, w, b):
    # ??

def d_model_db(X, w, b):
    # ??

def grad_fn(X, targets, predictions, w, b):
    # ??

In [ ]:
grad_fn(inputs, targets, prediccion, w_test, b_test)

# Entrenando

Ahora que tenemos los gradientes de los pesos y bias, podemos actualizar los parámetros del modelo para ir mejorando nuestras predicciones


In [ ]:
def training_loop(n_epochs, learning_rate, params, targets, X):
    # ??
            
    return params

In [ ]:
final_params = training_loop(
    n_epochs = 3000, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0, 0.0]), 
    targets = targets, 
    X = inputs)

final_params

In [ ]:
from matplotlib import pyplot as plt

preds = model(inputs, *final_params)  

plt.xlabel("Temperature (Unknown)")
plt.ylabel("Temperature (°Celsius)")
plt.plot(inputs.numpy(), preds.detach().numpy()) 
plt.plot(inputs.numpy(), targets.numpy(), 'o')
plt.show()

# Como se resuelve el mismo ejercicio cuando dejamos que Pytorch nos ayude?

Resuelva el mismo problema pero haciendo uso de Pytorch en su totalidad, defina w y b como un torch.parameter (requires_grad=True!!!) y haga uso de loss.backward() para computar los gradientes. 

In [ ]:
# En Pytorch
def training_loop(n_epochs, optimizer, params, X, targets):
  # ??

In [ ]:
import torch.optim as optim

params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate) # <1>

final_params = training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    params = params, # <1> 
    X = inputs,
    targets = targets)

In [ ]:
preds = model(inputs, *final_params)  

plt.xlabel("Temperature (Unknown)")
plt.ylabel("Temperature (°Celsius)")
plt.plot(inputs.numpy(), preds.detach().numpy()) 
plt.plot(inputs.numpy(), targets.numpy(), 'o')
plt.show()